# Word2Vec 사용하기

### gensim 패키지 이용

In [ ]:
from gensim.models import Word2Vec

#### gensim 설치 명령: pip install --upgrade gensim
#### gensim installation instructions: https://radimrehurek.com/gensim/install.html

#### Word2Vec 모델 학습시키기

In [ ]:
# 모델에 들어갈 training data 만들기
sentences = [['once', 'upon', 'a', 'time', 'there', 'was', 'an', 'old', 'sow', 'with', 'three', 'little', 'pig'],
             ['the', 'first', 'that', 'went', 'off', 'met', 'a', 'man', 'with', 'a', 'bundle', 'of', 'straw'],
             ['the', 'second', 'pig', 'met', 'a', 'man', 'with', 'a', 'bundle', 'of', 'furze'],
             ['the', 'third', 'little', 'pig', 'met', 'a', 'men', 'with', 'a', 'load', 'of', 'bricks']]

# 모델 학습시키기
model = Word2Vec(sentences, min_count=1) # default size = 100

In [ ]:
# 학습된 모델 확인하기
print(model)

In [ ]:
# 모델에 구축된 vocabulary 확인하기
words = list(model.wv.vocab)
print(words)

#### 단어 하나에 대한 임베딩 벡터 얻기

In [ ]:
print(model['pig'])

In [ ]:
print(model['little'])

In [ ]:
# 별도의 파일로 모델 저장하기
model.save('model.bin')

In [ ]:
# 저장해 둔 모델 불러오기
new_model = Word2Vec.load('model.bin')
print(new_model)

# 단어 임베딩 시각화하기

In [ ]:
# PCA를 이용하여 단어 벡터를 시각화
# PCA는 고차원의 단어 벡터를 2차원으로 줄여 시각화할 수 있는 형태로 만들어 줌 
from sklearn.decomposition import PCA
from matplotlib import pyplot

In [ ]:
# 학습된 word2vec 모델
# vocabulary에 들어 있는 단어 각각에 대해 100차원의 벡터가 생성되어 있음
X = model[model.wv.vocab]
print(X)

In [ ]:
# 이 벡터(여기서는 30x100)에 2차원 PCA 모델을 적용시켜 2차원으로 바꾸어 줌
pca = PCA(n_components=2)
result = pca.fit_transform(X)
print(result)

In [ ]:
# 2차원 프로젝션 결과를 산점도에 그릴 수 있음
# 각 단어는 모델에서 학습한 의미에 따라 적절한 위치에 그려지게 됨
pyplot.scatter(result[:, 0], result[:, 1])
words = list(model.wv.vocab)
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

----------------------------------------------------------------------------------------------------------------------

# Load Google’s Word2Vec Embedding

In [ ]:
# 미리 학습해 둔 word2vec 모형을 사용할 수 있음
# 구글에서 뉴스 데이터를 가지고 학습, 구축한 것이 대표적
# 300만 개 정도의 단어와 phrase로 구성, 300차원으로 표현

# url:
# https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing

path = '/Users/sana/Documents/'

from gensim.models import KeyedVectors
filename = path+'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [ ]:
# word2vec analogy
# calculate: (king - man) + woman = ?
result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

In [ ]:
# calculate: (Paris - France) + Korea = ?
result2 = model.most_similar(positive=['Paris', 'Korea'], negative=['France'], topn=1)
print(result2)

----------------------------------------------------------------------------------------------------------------------

# Load Stanford's GloVe Embedding

In [ ]:
# Global Vectors for Word Representation
# 스탠포드에서 만든 word embedding algorithm

# pre-trained vectors:
# http://ling.snu.ac.kr/class/cl_under1801/glove.6B.300d.txt.zip

# 이것을 gensim을 통해 word2vec의 형식으로 바꾸어 사용 가능

from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = path+'glove.6B.300d.txt'
word2vec_output_file = path+'glove.6B.300d.txt.word2vec'

# word2vec 형식으로 바꾸어 저장
glove2word2vec(glove_input_file, word2vec_output_file)

In [ ]:
from gensim.models import KeyedVectors

# load the Stanford GloVe model
filename = path+'glove.6B.300d.txt.word2vec'
model2 = KeyedVectors.load_word2vec_format(filename, binary=False)

In [ ]:
# calculate: (king - man) + woman = ?
result = model2.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

In [ ]:
# calculate: (Paris - France) + Korea = ?
result2 = model2.most_similar(positive=['paris', 'korea'], negative=['france'], topn=1)
print(result2)

----------------------------------------------------------------------------------------------------------------------

# FastText

In [ ]:
# 구글에서 개발한 Word2Vec을 기본으로 하되 부분단어들을 임베딩하는 기법
# 노이즈가 많은 데이터에 유리

from gensim.models import FastText
sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]

# initialize and train
model = FastText(sentences, min_count=1)
model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)
say_vector = model['say']  # get vector for word

print(say_vector.shape)
print(say_vector)

In [ ]:
# 모델을 학습할 때 사용한 코퍼스에 없는 단어라도
# FastText는 그 단어를 구성하기 위해 n-grams를 덧붙여서 벡터를 만들 수 있음

of_vector = model['of']  # get vector for out-of-vocab word

print(of_vector.shape)
print(of_vector)

In [ ]:
# Train a model and update vocab for online training

sentences_1 = [["cat", "say", "meow"], ["dog", "say", "woof"]]
sentences_2 = [["dude", "say", "wazzup!"]]

model = FastText(min_count=1)
model.build_vocab(sentences_1)
model.train(sentences_1, total_examples=model.corpus_count, epochs=model.iter)
model.build_vocab(sentences_2, update=True)
model.train(sentences_2, total_examples=model.corpus_count, epochs=model.iter)

----------------------------------------------------------------------------------------------------------------------

# 한국어 데이터 처리하기

In [ ]:
# Install KoNLPy
pip install konlpy

# http://konlpy.org/ko/latest/install/ 참조

#### 데이터 전처리: konlpy를 이용한 형태소 분석

In [2]:
import konlpy

from konlpy.tag import Komoran
from konlpy.tag import Twitter
from konlpy.tag import Hannanum
# from konlpy.tag import Mecab
# from konlpy.tag import Kkma

# initialize taggers
komo = Komoran()
twit = Twitter()
hann = Hannanum()

sent = '이것은 샘플 문장입니다.'

# Komoran parsing
print('Komoran:'+str(komo.pos(sent)))
# Twitter parsing
print('Twitter:'+str(twit.pos(sent)))
# Hannanum parsing
print('Hannanum:'+str(hann.pos(sent)))

Komoran:[('이것', 'NP'), ('은', 'JX'), ('샘플', 'NNG'), ('문장', 'NNG'), ('이', 'VCP'), ('ㅂ니다', 'EF'), ('.', 'SF')]
Twitter:[('이', 'Determiner'), ('것', 'Noun'), ('은', 'Josa'), ('샘플', 'Noun'), ('문장', 'Noun'), ('입니', 'Adjective'), ('다', 'Eomi'), ('.', 'Punctuation')]
Hannanum:[('이것', 'N'), ('은', 'J'), ('샘플', 'N'), ('문장', 'N'), ('이', 'J'), ('ㅂ니다', 'E'), ('.', 'S')]


In [22]:
sent2 = '이것은 샘플 문장입니닼ㅋㅋ'

# Komoran parsing
print('Komoran:'+str(komo.pos(sent2)))
# Twitter parsing
print('Twitter:'+str(twit.pos(sent2)))
# Hannanum parsing
print('Hannanum:'+str(hann.pos(sent2)))

Komoran:[('이것', 'NP'), ('은', 'JX'), ('샘플', 'NNG'), ('문장입니닼ㅋㅋ', 'NA')]
Twitter:[('이', 'Determiner'), ('것', 'Noun'), ('은', 'Josa'), ('샘플', 'Noun'), ('문장', 'Noun'), ('입니', 'Adjective'), ('닼', 'Noun'), ('ㅋㅋ', 'KoreanParticle')]
Hannanum:[('이것', 'N'), ('은', 'J'), ('샘플', 'N'), ('문장입니닼ㅋㅋ', 'N')]


#### Word2Vec 모델 학습시키기

In [ ]:
# 모델에 들어갈 training data 만들기

# 파일 읽어 오기
path = '/Users/sana/Documents/'
filename = path + 'KorNewsSample.txt'
f = open(filename, 'rU')
lines = f.readlines()
print(lines[:5])

In [ ]:
tagger = konlpy.tag.Komoran()

# konlpy로 형태소 분석하기
sentences = []
for line in lines:
    sentence = []
    pos = tagger.pos(line.strip())  # list of (형태소, 품사)
    for pair in pos:
        morpheme = pair[0]
        sentence.append(morpheme)
    sentences.append(sentence)
    
print(len(sentences))    
print(sentences[0])

In [ ]:
from gensim.models import Word2Vec

# 모델 학습시키기
model_ko = Word2Vec(sentences, min_count=1, size=300) 

In [ ]:
# 학습된 모델 확인하기
print(model_ko)

In [ ]:
# 모델에 구축된 vocabulary 확인하기
words = list(model_ko.wv.vocab)
print(len(words))
print(words)

#### 단어 하나에 대한 임베딩 벡터 얻기

In [ ]:
print(model_ko['국민'])

In [ ]:
print(model_ko['는'])

In [ ]:
print(model_ko['데이터'])  # unseen words

In [ ]:
# 별도의 파일로 모델 저장하기
model_ko.save('model_korean.bin')

In [ ]:
# 저장해 둔 모델 불러오기
new_model = Word2Vec.load('model_korean.bin')
print(new_model)

#### model similarity test

In [ ]:
print ("Similarity Test {한국/NNP+도쿄/NNP-서울/NNP}")
similars = model_ko.most_similar(positive=[u"한국", u"도쿄"], negative=[u"서울"], topn=10)
for word, value in similars:
    print(word, value)

In [ ]:
print ("Similarity Test {삼성}")
similars = model_ko.most_similar(positive=[u"삼성"], topn=10)
for word, value in similars:
    print(word, value)

In [ ]:
print ("Similarity Test {미래-가}")
similars = model_ko.most_similar(positive=[u"미래가"], topn=10)
for word, value in similars:
    print(word, value)

In [ ]:
print ("Similarity Test {미래}")
similars = model_ko.most_similar(positive=[u"미래"], topn=10)
for word, value in similars:
    print(word, value)

### 품사 정보를 포함시키는 경우

In [ ]:
tagger = konlpy.tag.Komoran()
# koNLPy로 형태소 분석하기
sentences = []
for line in lines:
    sentence = []
    pos = tagger.pos(line.strip())  # list of (형태소, 품사)
    for pair in pos:
        morpheme = pair[0]+'/'+pair[1] # 형태소/품사 형태
        sentence.append(morpheme)
    sentences.append(sentence)
    
print(len(sentences))    
print(sentences[0])

In [ ]:
# 모델 학습시키기
model_ko_2 = Word2Vec(sentences, min_count=1, size=300) 

In [ ]:
# 학습된 모델 확인하기
print(model_ko_2)

In [ ]:
# 모델에 구축된 vocabulary 확인하기
words = list(model_ko_2.wv.vocab)
print(len(words))
print(words)

#### 단어 하나에 대한 임베딩 벡터 얻기

In [ ]:
print(model_ko_2['국민/NNG'])

In [ ]:
print(model_ko_2['는/JX'])

In [ ]:
print(model_ko_2['데이터/NNG'])  # unseen words

In [ ]:
# 별도의 파일로 모델 저장하기
model_ko_2.save('model_korean_2.bin')

In [ ]:
# 저장해 둔 모델 불러오기
new_model = Word2Vec.load('model_korean_2.bin')
print(new_model)

#### model similarity test

In [ ]:
print ("Similarity Test {한국/NNP+도쿄/NNP-서울/NNP}")
similars = model_ko_2.most_similar(positive=[u"한국/NNP", u"도쿄/NNP"], negative=[u"서울/NNP"], topn=10)
for word, value in similars:
    print(word, value)

In [ ]:
print ("Similarity Test {삼성/NNP}")
similars = model_ko_2.most_similar(positive=[u"삼성/NNP"], topn=10)
for word, value in similars:
    print(word, value)

In [ ]:
print ("Similarity Test {미래/NNG}")
similars = model_ko_2.most_similar(positive=[u"미래/NNG"], topn=10)
for word, value in similars:
    print(word, value)

### 품사 정보를 이용하여 content word 필터링하기

In [ ]:
tagger = konlpy.tag.Komoran()
# koNLPy로 형태소 분석하기
sentences = []
for line in lines:
    sentence = []
    pos = tagger.pos(line.strip())  # list of (형태소, 품사)
    for pair in pos:
        if pair[1] in ['NNG', 'NNP', 'VV', 'VA']: # part-of-speech of content word
            morpheme = pair[0]+'/'+pair[1] # 형태소/품사 형태
            sentence.append(morpheme)
    sentences.append(sentence)
    
print(len(sentences))    
print(sentences[0])

In [ ]:
# 모델 학습시키기
model_ko_3 = Word2Vec(sentences, min_count=1, size=300) 

In [ ]:
# 학습된 모델 확인하기
print(model_ko_3)

In [ ]:
# 모델에 구축된 vocabulary 확인하기
words = list(model_ko_3.wv.vocab)
print(len(words))
print(words)

#### 단어 하나에 대한 임베딩 벡터 얻기

In [ ]:
print(model_ko_3['국민/NNG'])

In [ ]:
print(model_ko_3['는/JX'])

In [ ]:
# 별도의 파일로 모델 저장하기
model_ko_2.save('model_korean_3.bin')

In [ ]:
# 저장해 둔 모델 불러오기
new_model = Word2Vec.load('model_korean_3.bin')
print(new_model)

#### model similarity test

In [ ]:
print ("Similarity Test {한국/NNP+도쿄/NNP-서울/NNP}")
similars = model_ko_3.most_similar(positive=[u"한국/NNP", u"도쿄/NNP"], negative=[u"서울/NNP"], topn=10)
for word, value in similars:
    print(word, value)

In [ ]:
print ("Similarity Test {삼성/NNP}")
similars = model_ko_3.most_similar(positive=[u"삼성/NNP"], topn=10)
for word, value in similars:
    print(word, value)

In [ ]:
print ("Similarity Test {미래/NNG}")
similars = model_ko_3.most_similar(positive=[u"미래/NNG"], topn=10)
for word, value in similars:
    print(word, value)